In [ ]:
!pip install pyspark #installing pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 37 kB/s 
     |████████████████████████████████| 199 kB 50.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8755fd9c14b24085446059a9b64d3024bf8a2299c6ef082b422f53afc5c259a8
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession #importing pyspark

In [ ]:
# Create SparkSession 
spark = SparkSession.builder \
      .master("local[*]") \
      .appName("compression_formats") \
      .getOrCreate() 

In [ ]:
# Reading parquet file
df_userdata = spark.read.parquet("/datalab/userdata1.parquet")
df_userdata.show()

+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
|  registration_dttm| id|first_name|last_name|               email|gender|     ip_address|                 cc|             country| birthdate|   salary|               title|            comments|
+-------------------+---+----------+---------+--------------------+------+---------------+-------------------+--------------------+----------+---------+--------------------+--------------------+
|2016-02-03 07:55:29|  1|    Amanda|   Jordan|    ajordan0@com.com|Female|    1.197.201.2|   6759521864920116|           Indonesia|  3/8/1971| 49756.53|    Internal Auditor|               1E+02|
|2016-02-03 17:04:03|  2|    Albert|  Freeman|     afreeman1@is.gd|  Male| 218.111.175.34|                   |              Canada| 1/16/1968|150280.17|       Accountant IV|                    |
|2016-02-03 01:09:31|  3|

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Doing some processing with the file laoded (userdata1.parquet)
df_userdata.createOrReplaceTempView("userdata")
sqldf = spark.sql("SELECT avg(salary) as avg_salary, country \
                  FROM userdata \
                  WHERE salary >= 50000 \
                  GROUP BY country \
                  ORDER BY avg_salary DESC")
sqldf.show()
sqldf_initial_count = spark.sql("SELECT count(*) FROM userdata")
sqldf_initial_count.show()

+------------------+-------------------+
|        avg_salary|            country|
+------------------+-------------------+
|278260.82999999996|           Mongolia|
|         277545.53|Trinidad and Tobago|
|         275864.65|        El Salvador|
|         275527.02|            Lebanon|
|         271701.21| Dominican Republic|
|         266398.87|             Norway|
|266394.41500000004|          Nicaragua|
|          265985.0|            Croatia|
|         264459.67|              Haiti|
|         263412.02|            Liberia|
|         261618.16|      Faroe Islands|
|         259912.24|      Aland Islands|
|         256483.86|           Paraguay|
|         253657.82|         Uzbekistan|
|         253339.32|         Costa Rica|
|         248316.41|     Norfolk Island|
|         235757.92|             Gambia|
|         235606.76|           Kiribati|
|         233719.55|            Ecuador|
|         230699.71|            Bermuda|
+------------------+-------------------+
only showing top

In [ ]:
#Converting the original file to CSV,JSON and orc.
# df_userdata.write.format("CSV").mode("overwrite").save("userdata.csv")
df_userdata.coalesce(1).write.format("csv").mode("overwrite").save("userdata.csv")
df_userdata.coalesce(1).write.format("json").mode("overwrite").save("userdata.json")
df_userdata.coalesce(1).write.format("orc").mode("overwrite").save("userdata.orc")


In [ ]:
#Checking filisizes
import os

def get_file_path(dirname, format):
  index = len(format)
  for root, dirs, files in os.walk(dirname):
    for file in files:
        if  file[-index:] == format:
          return (dirname+file)

csv_file_path = get_file_path("./userdata.csv/", ".csv")
json_file_path = get_file_path("./userdata.json/", ".json")
orc_file_path = get_file_path("./userdata.orc/", ".orc")

print("the size for the file in format .csv is {} bytes".format(os.path.getsize(csv_file_path)))
print("the size for the file in format .json is {} bytes".format(os.path.getsize(json_file_path)))
print("the size for the file in format .orc is {} bytes".format(os.path.getsize(orc_file_path)))

the size for the file in format .csv is 149109 bytes
the size for the file in format .json is 304126 bytes
the size for the file in format .orc is 67880 bytes


In [ ]:
#Moving files to a different location
import shutil
csv_file_new_location = shutil.move(csv_file_path, ".")
json_file_new_location = shutil.move(json_file_path, ".")
orc_file_new_location = shutil.move(orc_file_path, ".")

In [ ]:
#Compressing files to gzip
import gzip
#compressing files to gzip
def file_to_zip(filename, file_format):
  compressed_filename = "file_"+file_format+".gz"
  with open(filename, 'rb') as f_in:
      with gzip.open(compressed_filename, 'wb') as f_out:
          shutil.copyfileobj(f_in, f_out)
  
  return compressed_filename
csv_filename_gzip = file_to_zip(csv_file_new_location, "csv")
json_filename_gzip = file_to_zip(csv_file_new_location, "json")
orc_filename_gzip = file_to_zip(csv_file_new_location, "orc")
print("the size for the csv file compressed in gzip format is {} bytes".format(os.path.getsize(csv_filename_gzip)))
print("the size for the json file compressed in gzip format is {} bytes".format(os.path.getsize(json_filename_gzip)))
print("the size for the orc file compressed in gzip format is {} bytes".format(os.path.getsize(orc_filename_gzip)))

the size for the csv file compressed in gzip format is 60721 bytes
the size for the json file compressed in gzip format is 60722 bytes
the size for the orc file compressed in gzip format is 60721 bytes


In [ ]:
#compressing files to bz2
import bz2
def file_to_bz2(filename, file_format):
  compressed_filename = "file_"+file_format+".bz2"
  with open(filename, mode="rb") as fin, bz2.open(compressed_filename, "wb") as fout:
    fout.write(fin.read())
  return compressed_filename
csv_filename_bz2 = file_to_bz2(csv_file_new_location, "csv")
json_filename_bz2 = file_to_bz2(csv_file_new_location, "json")
orc_filename_bz2 = file_to_bz2(csv_file_new_location, "orc")
print("the size for the csv file compressed in bz2 format is {} bytes".format(os.path.getsize(csv_filename_bz2)))
print("the size for the json file compressed in bz2 format is {} bytes".format(os.path.getsize(json_filename_bz2)))
print("the size for the orc file compressed in bz2 format is {} bytes".format(os.path.getsize(orc_filename_bz2)))

the size for the csv file compressed in bz2 format is 44884 bytes
the size for the json file compressed in bz2 format is 44884 bytes
the size for the orc file compressed in bz2 format is 44884 bytes


In [ ]:
#compressing files to zlib
import zlib
def file_to_zlib(filename, file_format):
  compressed_filename = "file_"+file_format+".zlib"
  with open(filename, mode="rb") as fin, open(compressed_filename, mode="wb") as fout:
      data = fin.read()
      compressed_data = zlib.compress(data, zlib.Z_BEST_COMPRESSION)
  return compressed_filename
csv_filename_zlib = file_to_zlib(csv_file_new_location, "csv")
json_filename_zlib = file_to_zlib(csv_file_new_location, "json")
orc_filename_zlib = file_to_zlib(csv_file_new_location, "orc")
print("the size for the csv file compressed in zlib format is {} bytes".format(os.path.getsize(csv_filename_zlib)))
print("the size for the json file compressed in zlib format is {} bytes".format(os.path.getsize(json_filename_zlib)))
print("the size for the orc file compressed in zlib format is {} bytes".format(os.path.getsize(orc_filename_zlib)))

the size for the csv file compressed in zlib format is 0 bytes
the size for the json file compressed in zlib format is 0 bytes
the size for the orc file compressed in zlib format is 0 bytes


In [ ]:
#Compressing file to format lzma
import lzma
lzc = lzma.LZMACompressor()
def file_to_lzma(filename, file_format):
  compressed_filename = "file_"+file_format+".lzma"
  with open(filename, mode="r") as fin, open(compressed_filename, "wb") as fout:
    for chunk in fin.read(1024):
        compressed_chunk = lzc.compress(chunk.encode("ascii"))
        fout.write(compressed_chunk)
    fout.write(lzc.flush())

  return compressed_filename
csv_filename_lzma = file_to_lzma(csv_file_new_location, "csv")
# json_filename_lzma = file_to_lzma(csv_file_new_location, "json")
# orc_filename_lzma = file_to_lzma(csv_file_new_location, "orc")

print("the size for the csv file compressed in lzma format is {} bytes".format(os.path.getsize(csv_filename_lzma)))
# print("the size for the json file compressed in lzma format is {} bytes".format(os.path.getsize(json_filename_lzma)))
# print("the size for the orc file compressed in lzma format is {} bytes".format(os.path.getsize(orc_filename_lzma)))

the size for the csv file compressed in lzma format is 664 bytes


In [ ]:
#Decompressing files compressed in format gzip
def decompress_gzip(compressed_filename, file_format):
  decompressed_filename = "decompressed_userdata_gz."+file_format
  fp = open(decompressed_filename, "wb")
  with gzip.open(compressed_filename, "rb") as f:
    bindata = f.read()
    fp.write(bindata)
    fp.close()
  return decompressed_filename
decompressed_csv = decompress_gzip("file_csv.gz", "csv")
decompressed_json = decompress_gzip("file_json.gz", "json")
decompressed_orc = decompress_gzip("file_orc.gz", "orc")

#checking filesizes
print("the size for the csv file decompressed from gzip format is {} bytes".format(os.path.getsize(decompressed_csv)))
print("the size for the json file decompressed from gzip format is {} bytes".format(os.path.getsize(decompressed_json)))
print("the size for the orc file decompressed from gzip format is {} bytes".format(os.path.getsize(decompressed_orc)))

#Counting rows for the decompressed csv file
df_userdata_decompressed_csv = spark.read.csv(decompressed_csv)
df_userdata_decompressed_csv.createOrReplaceTempView("userdata_decompressed_csv")
sqldf_userdata_decompressed_csv = spark.sql("SELECT count(*) FROM userdata_decompressed_csv")
print("count of the number of rows of the decompressed file in format CSV")
sqldf_userdata_decompressed_csv.show()

#Counting rows for the decompressed json file
df_userdata_decompressed_json = spark.read.json(decompressed_json)
df_userdata_decompressed_json.createOrReplaceTempView("userdata_decompressed_json")
sqldf_userdata_decompressed_json = spark.sql("SELECT count(*) FROM userdata_decompressed_json")
print("count of the number of rows of the decompressed file in format json")
sqldf_userdata_decompressed_json.show()

#Counting rows for the decompressed orc file
# df_userdata_decompressed_orc = spark.read.orc(decompressed_orc)
# df_userdata_decompressed_orc.createOrReplaceTempView("userdata_decompressed_orc")
# sqldf_userdata_decompressed_orc = spark.sql("SELECT count(*) FROM userdata_decompressed_orc")
# print("count of the number of rows of the decompressed file in format orc")
# sqldf_userdata_decompressed_orc.show()


the size for the csv file decompressed from gzip format is 149109 bytes
the size for the json file decompressed from gzip format is 149109 bytes
the size for the orc file decompressed from gzip format is 149109 bytes
count of the number of rows of the decompressed file in format CSV
+--------+
|count(1)|
+--------+
|    1000|
+--------+

count of the number of rows of the decompressed file in format json
+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [ ]:
# Decompressing from bz2
def decompress_bz2(compressed_filename, file_format):
  decompressed_filename = "decompressed_userdata_bz2."+file_format
  zipfile = bz2.BZ2File(compressed_filename) # open the file
  data = zipfile.read() # get the decompressed data
  open(decompressed_filename, 'wb').write(data) # write a uncompressed file
  return decompressed_filename
decompressed_csv = decompress_bz2("file_csv.bz2", "csv")
decompessed_json = decompress_bz2("file_json.bz2", "json")
decompressed_orc = decompress_bz2("file_orc.bz2", "orc")

#checking filesizes
print("the size for the csv file decompressed from bz2 format is {} bytes".format(os.path.getsize(decompressed_csv)))
print("the size for the json file decompressed from bz2 format is {} bytes".format(os.path.getsize(decompessed_json)))
print("the size for the orc file decompressed from bz2 format is {} bytes".format(os.path.getsize(decompressed_orc)))

#Counting rows for the decompressed csv file
df_userdata_decompressed_csv = spark.read.csv(decompressed_csv)
df_userdata_decompressed_csv.createOrReplaceTempView("userdata_decompressed_csv")
sqldf_userdata_decompressed_csv = spark.sql("SELECT count(*) FROM userdata_decompressed_csv")
print("count of the number of rows of the decompressed file in format CSV")
sqldf_userdata_decompressed_csv.show()

#Counting rows for the decompressed json file
df_userdata_decompressed_json = spark.read.json(decompressed_json)
df_userdata_decompressed_json.createOrReplaceTempView("userdata_decompressed_json")
sqldf_userdata_decompressed_json = spark.sql("SELECT count(*) FROM userdata_decompressed_json")
print("count of the number of rows of the decompressed file in format json")
sqldf_userdata_decompressed_json.show()


the size for the csv file decompressed from bz2 format is 149109 bytes
the size for the json file decompressed from bz2 format is 149109 bytes
the size for the orc file decompressed from bz2 format is 149109 bytes
count of the number of rows of the decompressed file in format CSV
+--------+
|count(1)|
+--------+
|    1000|
+--------+

count of the number of rows of the decompressed file in format json
+--------+
|count(1)|
+--------+
|    1000|
+--------+



In [ ]:
# #Decompressing from zlib
# def decompress_zlib(compressed_filename, file_format):
#   decompressed_filename = "decompressed_userdata_zlib."+file_format
#   str_object1 = open(compressed_filename, 'rb').read()
#   str_object2 = zlib.decompress(str_object1)
#   f = open(decompressed_filename, 'wb')
#   f.write(str_object2)
#   f.close()
#   return decompressed_filename
# decompressed_csv = decompress_zlib("file_csv.zlib", "csv")
# decompessed_json = decompress_zlib("file_json.zlib", "json")
# decompressed_orc = decompress_zlib("file_orc.zlib", "orc")

# #checking filesizes
# print("the size for the csv file decompressed from zlib format is {} bytes".format(os.path.getsize(decompressed_csv)))
# print("the size for the json file decompressed from zlib format is {} bytes".format(os.path.getsize(decompessed_json)))
# print("the size for the orc file decompressed from zlib format is {} bytes".format(os.path.getsize(decompressed_orc)))



error: ignored

In [ ]:
# #Decompressing from lzma
# def decompress_lzma(compressed_filename, file_format):
#   decompressed_filename = "decompressed_userdata_lzma."+file_format
#   lzd = lzma.LZMADecompressor()
#   compressed_filename = "file_"+file_format+".lzma"
#   with lzma.open(compressed_filename, mode="rb") as f:
#     file_content = f.read()
#     print(file_content)
  
#   # return decompressed_filename
# decompressed_csv = decompress_lzma("file_csv.lzma", "csv")
# # decompessed_json = decompress_bz2("file_json.bz2", "json")
# # decompressed_orc = decompress_bz2("file_orc.bz2", "orc")

# #checking filesizes
# # print("the size for the csv file decompressed from lzma format is {} bytes".format(os.path.getsize(decompressed_csv)))
# # print("the size for the json file decompressed from bz2 format is {} bytes".format(os.path.getsize(decompessed_json)))
# # print("the size for the orc file decompressed from bz2 format is {} bytes".format(os.path.getsize(decompressed_orc)))



b''


# New Section

In [ ]:
df = spark.read.csv('/content/diabetes.csv', header = True) #Read a csv file
df.printSchema()

root
 |-- Pregnancies: string (nullable = true)
 |-- Glucose: string (nullable = true)
 |-- BloodPressure: string (nullable = true)
 |-- SkinThickness: string (nullable = true)
 |-- Insulin: string (nullable = true)
 |-- BMI: string (nullable = true)
 |-- DiabetesPedigreeFunction: string (nullable = true)
 |-- Age: string (nullable = true)
 |-- Outcome: string (nullable = true)



In [ ]:
df.show(5) #the first 5 rows

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
only showing top 5 rows



In [ ]:
insulin_10 = df.filter(df.Insulin > 10) #process 1
insulin_10.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|
|          2|    197|           70|           45|    543|30.5|                   0.158| 53|      1|
|          1|    189|           60|           23|    846|30.1|                   0.398| 59|      1|
|          5|    166|           72|           19|    175|25.8|                   0.587| 51|      1|
|          0|    118|           84|           47|    230|45.8|                   0.551| 31|      1|


In [ ]:
preg_count = df.groupby("Pregnancies").count()
preg_count.show()

In [ ]:
# converting to parquet
file_path = "/content/resultparquet"
preg_count.write.mode('append').parquet(file_path)

In [ ]:
# checking file size of snappy comp. tech
import os
parquet_loc = ('/content/resultparquet/part-00000-0d248719-4191-4299-af23-e9ab2088a2d4-c000.snappy.parquet')
file_stat = os.stat(parquet_loc)
print(file_stat.st_size)

855


In [ ]:
#this converts to gzip
gzip_file_path = parquet_loc
preg_count.write\
    .option("delimiter", "\t")\
    .option("codec", "org.apache.hadoop.io.compress.GzipCodec")\
    .save(
        path='/content/resultgzip',
        format='parquet',
        header=True,
        emptyValue='',
        compression="gzip"
)

In [ ]:
# checking file size of gzip
import os
file_stat = os.stat('/content/resultgzip/part-00000-8766c35e-611a-4328-a453-37f352af500a-c000.gz.parquet')
print(file_stat.st_size)

829


In [ ]:
#gzip 
gzip_file_path = "/content/resultgzip/part-00000-8766c35e-611a-4328-a453-37f352af500a-c000.gz.parquet"
preg_count.write\
    .option("delimiter", "\t")\
    .option("codec", "org.apache.hadoop.io.compress.SnappyCodec")\
    .save(
        path='/content/gzip_comp_decompressed1',
        format='parquet',
        header=True,
        emptyValue='',
        compression="snappy"
)
    


In [ ]:
# checking file size of decompressed gzip
import os
file_stat = os.stat('/content/gzip_comp_decompressed1/part-00000-b1caa7df-6759-42b0-b7a5-584b942a66c8-c000.snappy.parquet')
print(file_stat.st_size)

855
